<a href="https://colab.research.google.com/github/AgustinCocciardi/TP-Ciencia-de-Datos/blob/main/TP_EntregableVirus_Pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Examen Práctico

#### 01-3900 | Ciencia de datos | 2024

Alumno: Cocciardi Agustin Ignacio

## Enunciado

Se tienen un dataset con datos de pacientes internados en un hospital (TP_Virus_Alumnos.csv). La clase de interes (1) refiere a la presencia de un virus. El virus tiene normalmente una gravedad leve/baja y el tratamiento suele ser invasivo. Datos como nombre y apellido han sido eliminados y los valores tanto en sangre (BLD), hormonales u otros análisis sobre reactivos han sido alterados en sus valores para preservar la privacidad. Se aclara que no se ha modificado su capacidad predictiva (Si es que la tienen).


Para su conocimiento: </BR>
Datos generales de Edad, Peso, Altura y condición laboral (Activo, Pasivo etc).
Datos medidos en hospital:</BR>
BLD: Sangre</BR>
LVL: Hormonales</BR>
REC: Otros análisis</BR>

Se pide obtener con los datos disponibles el mejor modelo posible que prediga la presencia o ausencia del virus.
Dado que el tratamiento es invasivo y la grevedad es moderada se requiere "atrapar" tantos "1" como sea posible y minimizar los falsos positivos para evitar que reciban un tratamiento de estas caracteristicas personas que no presentan el virus. Intente obtener el mejor modelo que maximice la métrica que considere correspondiente.



## Como desarrollar el exámen

A partir del dataset realice todas las acciones para poder llegar al mejor modelo, explique brevemente en los fundamentos de sus transformaciones o acciones en general.

La nota derivará de: </BR>
1.La calidad de la clasificación realizada</BR>
2.La fundamentación de los pasos realizados</BR>
3.Lo sencillo de llevar a producción el desarrollo</BR>



Los docentes evaluaran su clasificador utilizando un conjunto de datos del dataset "fuera de la caja" (out of the box, al que usted no tiene acceso). Para minimizar la posible diferencia entre su medición y la medición del docente recuerde y aplique conceptos de test, validación cruzada y evite los errores comunes de sesgo de selección y fuga de datos (PPT/Pdf Árboles de clasificación) o  Sklearn "10. Common pitfalls and recommended practices" disponible en "https://scikit-learn.org/stable/common_pitfalls.html"   

Al final del notebook encontrará un bloque de código que lee la muestra adicional (a la que usted no tiene acceso) si PRODUCCION==True, en caso contrario solo lee una submuestra del conjunto original para validar que el código funciona. Desarrolle el notebook como considere, para finalmente asignar el mejor clasificador que usted haya obtenido remplazando en f_clf = None, None por su clasificador. Implemente todas las transformaciones entre esa lí­nea y la predición final (Evitando la fuga de datos). Ver TP_AutomatizarTransformaciones.ipynb

En materiales del MIEL se adjunta un notebook que propone algunas ideas para automatizar el proceso.

## Evaluacion final - Docente + Alumno

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

PRODUCCION = False
best_clf = None #Asignar aqui el mejor clasificador posible (previamente entrenado)

#Leemos el dataset de evaluacion, simulando produccion
if PRODUCCION==False:
    df = pd.read_csv("TP_Virus_Alumnos.csv")
    _, df = train_test_split(df, test_size=0.3, random_state=42)
else:
    df = pd.read_csv("TP_Virus_Evaluacion.csv")
#Dividimos en target y predictoras

X_prod = df.drop("target", axis=1)
y_prod = df["target"]

#Transformaciones


#Evaluacion final
y_pred = best_clf.predict(X_prod)
print(classification_report(y_prod, y_pred))

FileNotFoundError: [Errno 2] No such file or directory: 'TP_Virus_Alumnos.csv'

# Desarrollo del TP

# Importo bibliotecas a usar

In [2]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_classification
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer

# Clases para el Pipeline

Clase para aplicar codificación Onehot

In [3]:
class ColOneHot(BaseEstimator, TransformerMixin):
    def __init__(self, encoder=None, columns=[]):
        super().__init__()
        self.encoder = encoder
        self.columns = columns

    def fit(self, X, y=None):
        self.encoder.fit(X[self.columns])
        return self

    def get_feature_names_out(self):
        return self.get_feature_names_out()

    def  transform(self, X):
        Xc = X.copy()
        Xc.loc[:, self.columns] = self.encoder.transform(Xc[self.columns])
        return Xc

Clase para aplicar Imputer

In [4]:
#Imputador con columnas, estrategia por composicion
class ColImputer(BaseEstimator, TransformerMixin):
    def __init__(self, imputer=SimpleImputer(), columns=[]):
        super().__init__()
        self.imputer = imputer
        self.columns = columns

    def fit(self, X, y=None):
        self.imputer.fit(X[self.columns])
        return self

    def get_feature_names_out(self):
        return self.imputer.get_feature_names_out()

    def  transform(self, X):
        Xc = X.copy()
        Xc.loc[:, self.columns] = self.imputer.transform(Xc[self.columns])
        return Xc

Clase para aplicar Scaler

In [5]:
class ColScaler(BaseEstimator, TransformerMixin):
    def __init__(self, columns=[], scaler=StandardScaler()):
        self.scaler = scaler
        self.columns = columns
        pass
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns])
        return self
    def get_feature_names_out(self):
        return None
    def  transform(self, X):
        x_copy = X.copy()
        x_copy[self.columns] = self.scaler.transform(x_copy[self.columns])
        return x_copy

Clase para imputar outliers (usando método de cuantiles)

In [7]:
class OutlierImputer(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None, threshold=1.5):
        self.columns = columns
        self.threshold = threshold

    def fit(self, X, y=None):
        if self.columns is None:
            raise ValueError("No se han especificado columnas para imputar los outliers.")
        self.quantiles_ = {}
        for col in self.columns:
            self.quantiles_[col] = np.nanpercentile(X[col], [25, 75])
        return self

    def transform(self, X):
        if self.columns is None:
            raise ValueError("No se han especificado columnas para imputar los outliers.")
        for col in self.columns:
            column_data = X[col]
            lower_quantile, upper_quantile = self.quantiles_[col]
            iqr = upper_quantile - lower_quantile
            lower_bound = lower_quantile - self.threshold * iqr
            upper_bound = upper_quantile + self.threshold * iqr
            imputed_column = np.where(
                (column_data < lower_bound) | (column_data > upper_bound),
                np.nanmedian(column_data),  # Puedes cambiar esto por otra estrategia de imputación si lo deseas
                column_data
            )
            X[col] = imputed_column
        return X

Clase para imputar nulos (con media, mediana o moda)

In [9]:
class NullImputer(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None, strategy='mean'):
        self.columns = columns
        self.strategy = strategy

    def fit(self, X, y=None):
        if self.columns is None:
            raise ValueError("No se han especificado columnas para imputar los valores nulos.")
        self.impute_values_ = {}
        for col in self.columns:
            if self.strategy == 'mean':
                self.impute_values_[col] = X[col].mean()
            elif self.strategy == 'median':
                self.impute_values_[col] = X[col].median()
            elif self.strategy == 'mode':
                self.impute_values_[col] = X[col].mode().iloc[0]  # Moda puede devolver varios valores, tomamos el primero
            else:
                raise ValueError("Estrategia de imputación no válida. Las opciones son 'mean', 'median' o 'mode'.")
        return self

    def transform(self, X):
        if self.columns is None:
            raise ValueError("No se han especificado columnas para imputar los valores nulos.")
        for col in self.columns:
            X[col].fillna(self.impute_values_[col], inplace=True)
        return X

Clase para eliminar columnas

In [10]:
class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if self.columns is None:
            return X
        else:
            return X.drop(columns=self.columns)

# Exploracion de datos

In [12]:
df = pd.read_csv("TP_Virus_Alumnos.csv")
df

,Edad,Genero,Peso,Laboral,hijos,BLD01,REC1,REC2,REC3,REC4,REC5,BLD02,BLD03,LVL,target
0,9.0,Femenino,36.410602,Inactivo,0,3.139714,6.929112,63.712640,73.880613,15.087239,90.400193,2.015331,1.633836,51.411632,0
1,58.0,Masculino,115.507184,Inactivo,4,2.953914,4.660136,64.582610,73.669319,14.503106,90.817947,0.973317,-0.439358,1000000.000000,0
2,74.0,Masculino,81.249035,Activo,3,1.365200,3.005861,63.613454,75.103984,14.077988,91.741638,3.243344,0.380413,1000000.000000,1
3,NaN,Masculino,77.985589,No_declara,0,2.329285,4.344642,64.433115,73.402436,15.583846,91.741709,3.188290,-0.226046,NaN,0
4,1.0,Femenino,13.446208,Inactivo,0,-0.062975,3.938741,65.122602,75.007982,16.961593,90.668625,2.729202,0.589698,21.627303,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4195,45.0,NaN,90.221760,Activo,1,1.660127,2.720841,64.551161,74.409061,15.196857,90.130661,0.936022,0.292744,112.514342,0
4196,9.0,Masculino,37.949680,Inactivo,0,2.001270,3.892882,64.271080,71.905067,15.192511,89.164612,2.887523,0.694750,1000000.000000,1
4197,24.0,Masculino,72.322983,Activo,1,1.598071,5.566376,65.217362,70.912155,15.975612,89.835998,-0.490912,1.572128,100.452789,1
4198,16.0,NaN,56.321757,Inactivo,0,1.804688,3.494108,61.385971,74.595457,14.696368,91.129958,1.937261,0.428380,82.772516,0


Reviso de qué tipo son los datos

In [13]:
df.dtypes

Edad       float64
Genero      object
Peso       float64
Laboral     object
hijos        int64
BLD01      float64
REC1       float64
REC2       float64
REC3       float64
REC4       float64
REC5       float64
BLD02      float64
BLD03      float64
LVL        float64
target       int64
dtype: object

Veo que la mayoría de datos son numéricos, con excepción de Genero y Laboral, que podrían ser considerados como columnas categóricas.

Analizo el porcentaje de nulos de cada columa.

In [14]:
round((df.isnull().sum()/df.shape[0])*100,2)

Edad       19.57
Genero     40.05
Peso        0.00
Laboral     0.00
hijos       0.00
BLD01       0.00
REC1        0.00
REC2        0.00
REC3        0.00
REC4        0.00
REC5        0.00
BLD02       0.00
BLD03       0.00
LVL        12.55
target      0.00
dtype: float64

Veo que las columnas Edad, Genero y LVL poseen valores nulos. En el caso de Edad y LVL, el porcentaje es bajo, por lo que podría imputar sus valores nulos, eliminar las columnas o decantarme por eliminar las filas donde esas columnas cueten con valores nulos.

Para la columna Genero, el valor es muy alto. Si elimino los registros en null de esa columna voy a perder el 40% de mi dataset. Tengo que, o bien imputar sus valores, o eliminar la columna.

Considero que la columna Genero no es significativa en lo que se refiere a decidir si un paciente tiene un virus o no. La vería importante si estuviéramos analizando un caso de disparidad salarial. Al no considerarla importante para este dataset, la eliminaré.

In [15]:
column_dropper = ColumnDropper(columns=["Genero"])
df = column_dropper.fit_transform(df)
df

,Edad,Peso,Laboral,hijos,BLD01,REC1,REC2,REC3,REC4,REC5,BLD02,BLD03,LVL,target
0,9.0,36.410602,Inactivo,0,3.139714,6.929112,63.712640,73.880613,15.087239,90.400193,2.015331,1.633836,51.411632,0
1,58.0,115.507184,Inactivo,4,2.953914,4.660136,64.582610,73.669319,14.503106,90.817947,0.973317,-0.439358,1000000.000000,0
2,74.0,81.249035,Activo,3,1.365200,3.005861,63.613454,75.103984,14.077988,91.741638,3.243344,0.380413,1000000.000000,1
3,NaN,77.985589,No_declara,0,2.329285,4.344642,64.433115,73.402436,15.583846,91.741709,3.188290,-0.226046,NaN,0
4,1.0,13.446208,Inactivo,0,-0.062975,3.938741,65.122602,75.007982,16.961593,90.668625,2.729202,0.589698,21.627303,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4195,45.0,90.221760,Activo,1,1.660127,2.720841,64.551161,74.409061,15.196857,90.130661,0.936022,0.292744,112.514342,0
4196,9.0,37.949680,Inactivo,0,2.001270,3.892882,64.271080,71.905067,15.192511,89.164612,2.887523,0.694750,1000000.000000,1
4197,24.0,72.322983,Activo,1,1.598071,5.566376,65.217362,70.912155,15.975612,89.835998,-0.490912,1.572128,100.452789,1
4198,16.0,56.321757,Inactivo,0,1.804688,3.494108,61.385971,74.595457,14.696368,91.129958,1.937261,0.428380,82.772516,0


Las columnas de Edad y LVL cuentan con valores nulos, y puedo ver además que LVL posee outliers. Por lo que voy a tener que imputar ambas columnas (en el caso de Edad, para completar nulos; y en el caso de LVL, para completar nulos y eliminar los outliers)

Imputo los nulos de ambas columnas.

In [16]:
null_imputer = NullImputer(columns=["Edad","LVL"],strategy='median')
df = null_imputer.fit_transform(df)
df

,Edad,Peso,Laboral,hijos,BLD01,REC1,REC2,REC3,REC4,REC5,BLD02,BLD03,LVL,target
0,9.0,36.410602,Inactivo,0,3.139714,6.929112,63.712640,73.880613,15.087239,90.400193,2.015331,1.633836,51.411632,0
1,58.0,115.507184,Inactivo,4,2.953914,4.660136,64.582610,73.669319,14.503106,90.817947,0.973317,-0.439358,1000000.000000,0
2,74.0,81.249035,Activo,3,1.365200,3.005861,63.613454,75.103984,14.077988,91.741638,3.243344,0.380413,1000000.000000,1
3,38.0,77.985589,No_declara,0,2.329285,4.344642,64.433115,73.402436,15.583846,91.741709,3.188290,-0.226046,108.636906,0
4,1.0,13.446208,Inactivo,0,-0.062975,3.938741,65.122602,75.007982,16.961593,90.668625,2.729202,0.589698,21.627303,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4195,45.0,90.221760,Activo,1,1.660127,2.720841,64.551161,74.409061,15.196857,90.130661,0.936022,0.292744,112.514342,0
4196,9.0,37.949680,Inactivo,0,2.001270,3.892882,64.271080,71.905067,15.192511,89.164612,2.887523,0.694750,1000000.000000,1
4197,24.0,72.322983,Activo,1,1.598071,5.566376,65.217362,70.912155,15.975612,89.835998,-0.490912,1.572128,100.452789,1
4198,16.0,56.321757,Inactivo,0,1.804688,3.494108,61.385971,74.595457,14.696368,91.129958,1.937261,0.428380,82.772516,0


Ahora imputo los outliers de LVL

In [17]:
outlier_imputer = OutlierImputer(columns=["LVL"])
df = outlier_imputer.fit_transform(df)
df

,Edad,Peso,Laboral,hijos,BLD01,REC1,REC2,REC3,REC4,REC5,BLD02,BLD03,LVL,target
0,9.0,36.410602,Inactivo,0,3.139714,6.929112,63.712640,73.880613,15.087239,90.400193,2.015331,1.633836,51.411632,0
1,58.0,115.507184,Inactivo,4,2.953914,4.660136,64.582610,73.669319,14.503106,90.817947,0.973317,-0.439358,108.636906,0
2,74.0,81.249035,Activo,3,1.365200,3.005861,63.613454,75.103984,14.077988,91.741638,3.243344,0.380413,108.636906,1
3,38.0,77.985589,No_declara,0,2.329285,4.344642,64.433115,73.402436,15.583846,91.741709,3.188290,-0.226046,108.636906,0
4,1.0,13.446208,Inactivo,0,-0.062975,3.938741,65.122602,75.007982,16.961593,90.668625,2.729202,0.589698,108.636906,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4195,45.0,90.221760,Activo,1,1.660127,2.720841,64.551161,74.409061,15.196857,90.130661,0.936022,0.292744,112.514342,0
4196,9.0,37.949680,Inactivo,0,2.001270,3.892882,64.271080,71.905067,15.192511,89.164612,2.887523,0.694750,108.636906,1
4197,24.0,72.322983,Activo,1,1.598071,5.566376,65.217362,70.912155,15.975612,89.835998,-0.490912,1.572128,100.452789,1
4198,16.0,56.321757,Inactivo,0,1.804688,3.494108,61.385971,74.595457,14.696368,91.129958,1.937261,0.428380,82.772516,0
